### Install Package

In [1]:
%pip install pip --upgrade

  Using cached pip-22.0.4-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
Note: you may need to restart the kernel to use updated packages.    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4



In [1]:
%pip install numpy --upgrade

     ---------------------------------------- 14.7/14.7 MB 3.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.22.3 which is incompatible.


In [1]:
%pip install pandas --upgrade

Note: you may need to restart the kernel to use updated packages.  Downloading pandas-1.4.1-cp39-cp39-win_amd64.whl (10.5 MB)
     --------------------------------------- 10.5/10.5 MB 58.2 kB/s eta 0:00:00

  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.4
    Uninstalling pandas-1.3.4:
      Successfully uninstalled pandas-1.3.4


In [1]:
%pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=f55d60051a7745a6d0d78babf5ebf62a7f5d4e11467dcb040ce99868b220dae5
  Stored in directory: c:\users\dochanh\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install xgboost

  Using cached xgboost-1.5.2-py3-none-win_amd64.whl (106.6 MB)
Note: you may need to restart the kernel to use updated packages.


### Start

In [1]:
import pandas as pd
import numpy as np
import xgboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pickle
import warnings

warnings.filterwarnings('ignore')
np.random.seed(0)

C:\Users\dochanh\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [9]:
# Data Set, Feature & Label
dataset = pd.read_csv('D:/dataset/PCM_group1.csv') #encoding="UTF16"
#dataset.dropna(inplace=True) # 결측치가 존재하는 row 삭제

# Data Preprocessing
x = dataset.filter(regex='avg -|std -')

# yield - log scaling
org_y = dataset.filter(regex='yield')
y = 100.1 - org_y
y = np.log(y)
dataset['log_yield'] = y

In [10]:
dataset.shape

(233, 213)

### Scaling, Outlier, Customizing obj&eval

In [ ]:
# 이상치 제거
def remove_out(dataframe, remove_col):
    dff = dataframe
    for k in remove_col:
        level_1q = dff[k].quantile(0.25)
        level_3q = dff[k].quantile(0.75)
        IQR = level_3q - level_1q
        rev_range = 6 # 제거 범위 조절 변수
        dff = dff[(dff[k] <= level_3q + (rev_range*IQR)) & (dff[k] >= level_1q - (rev_range*IQR))]
        dff = dff.reset_index(drop=True)
    return dff

X_train.shape
col_list = list(x.columns)
non_outlier = remove_out(dataset, col_list)
non_outlier.shape

In [ ]:
# 0-1 Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x)
scaled_x = scaler.transform(x)
scaled_x = pd.DataFrame(scaled_x)

from pickle import dump
dump(scaler, open('C:/', 'wb'))

from pickle import load
mm_scaler = load(open('', 'rb'))

In [ ]:
# objective function, eval_metric -> 오차 네 제곱으로 Customizing
def custom_objective(labels, preds):
    grad = 4*(preds - labels)**3
    hess = 12*(preds - labels)**2 #grad를 미분한것이 hess
    return grad, hess

def custom_evaluation(preds, dtrain):
    labels = dtrain.get_label()
    return "custom_error", ((np.subtract(preds, labels))**4).mean()

### Modeling

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0) #scaled_x

In [12]:
# default, baseline
xgb_model = xgboost.XGBRegressor()
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [31]:
# Model Design & learning
xgb_model = xgboost.XGBRegressor(
    n_estimators=1500,
    learning_rate=0.08,
    gamma=0.5,
    subsample=0.5,
    colsample_bytree=0.5,
    max_depth=6,
    verbosity=1)
    # objective = custom_objective
    # eval_metric = custom_evaluation
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, enable_categorical=False,
             gamma=0.5, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.08, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1500, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=1)

In [ ]:
xgb_model.save_model('C:/')

In [13]:
# custom score
def accuracy(predictions, real_y):
    residual = abs(predictions - real_y)
    good = residual <= 1.5
    good = good.loc[good==True]
    bad = residual > 1.5
    bad = bad.loc[bad == True]
    score = len(good)/(len(good)+len(bad))
    return score*100

In [33]:
predictions = xgb_model.predict(X_test)
real_predictions = np.exp(predictions)
real_predictions = 100.1 - real_predictions

real_y_test = np.exp(y_test)
real_y_test = 100.1  - real_y_test

r_sq = xgb_model.score(X_train, y_train)
ev_score = explained_variance_score(real_predictions, real_y_test)
mse = mean_squared_error(real_y_test, real_predictions)
rmse = mse**(1/2.0)
mae = mean_absolute_error(real_y_test, real_predictions)
#acc = accuracy(real_y_test, predictions)

print(r_sq, ev_score, mse, rmse, mae, sep='\n')

0.5663247133597713
-2.9901889194550857
593.534230305082
24.362557958988667
17.78959092625809


### Parameter Tuning

In [27]:
xgb_parameter_bound = {
    'gamma':(0,5),
    'max_depth':(4,7),
    'subsample':(0.5,1),
    'learning_rate':(0.01,0.2),
    'colsample_bytree':(0.5,1)
}

In [26]:
def xgb_bo(gamma, max_depth, subsample, learning_rate, colsample_bytree):
    xgb_params = {
        'gamma':int(round(gamma)),
        'max_depth':int(round(max_depth)),
        'subsample':int(round(subsample)),
        'learning_rate':int(round(learning_rate)),
        'colsample_bytree':int(round(colsample_bytree))
    }
    xgb = xgboost.XGBRegressor(**xgb_params, n_estimators=1500)
    
    X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.1)
    xgb.fit(X_train, y_train)
    
    predictions = xgb.predict(X_valid)
    
    score = -mean_squared_error(y_valid, predictions)
    
    return score

In [ ]:
from bayes_opt import BayesianOptimization

BO_xgb = BayesianOptimization(f=xgb_bo, pbounds=xgb_parameter_bound, random_state=0)
BO_xgb.maximize(init_points=5, n_iter=10)

### New Data Prediction

In [ ]:
# test 결과값

model = xgboost.XGBRegressor()
model.load_model("C:/")

testdata = pd.read_csv('C:/')
testdata.dropna(inplace=True)
test = testdata.filter(regex='avg -|std -')
scaled_test = mm_scaler.transform(test)

testdata['Used Model'] = 'Program_Name'
testdata['retrained flag'] = 0

predictions = model.predict(scaled_test)
real_predictions = np.exp(predictions)
real_predictions = 100.1 - real_predictions

feature_importance = pd.DataFrame({"Feature": test.columns,
                                   "Importance": model.feature_importances_})

df_prediction = pd.DataFrame(real_predictions, columns=['predict'])
results_set = pd.concat([testdata['Used Model'], testdata['Lot'], testdata['Wafer'], df_prediction, testdata['retrained flag']], axis=1)
results_set.insert(4, 'yield (%)','')